In [1]:
%%capture
%run full_setup.py

In [2]:
from tfidf_corpus_dictionary import get_tfidf_tokendocs_corpus_dict
from gensim.models import LdaModel, LsiModel, CoherenceModel
from sklearn.decomposition import NMF, PCA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
import numpy as np
from scipy import sparse

With this function we get various objects needed for modelling:
1. TFIDF matrix as input data, with specified parameters
2. feature names as the words retained with TFIDF
3. tokenized documents, a list of lists, where the inner lists contain the tokens for each document
4. corpus, gensim object needed for modelling with that package
4. dictionary, gensim object containing informations on the words of the corpus and their positions

In [3]:
tfidf_matrix, feature_names, tokenized_docs, corpus, dictionary = get_tfidf_tokendocs_corpus_dict(df, max_df=0.5, min_df=5, max_features=5000)

Now we'll evaluate different topic models based on Coherence score.

Coherence is a metric used to evalute topics quality. The higher the coherence, the better the model did in creating the topics.

For every model we'll use a function to retrieve coherence for different numbers of topics (5, 10, 15, 20, 50). This information will be used to evaluate how the models performed as the number of topics changes

In [4]:
from coherence_topics import coherence_topics

In [9]:
evaluation = dict()
models = ['LDA', 'LSA', 'NMF', 'PCA', 'RP']

for mod in models:
    metrics = coherence_topics(model_name=mod, corpus=corpus, dictionary=dictionary,
                               texts=tokenized_docs, feature_names=feature_names, tfidf=tfidf_matrix)
    evaluation[mod] = metrics

In [10]:
evaluation['LDA']

[(5, 0.545979304624336),
 (10, 0.5035821857797423),
 (15, 0.5275933069581035),
 (20, 0.5134451552900542),
 (50, 0.46410568089530935)]

In [11]:
evaluation['LSA']

[(5, 0.38657061293889755),
 (10, 0.46671223720227306),
 (15, 0.4080013163682798),
 (20, 0.37904110899562504),
 (50, 0.3633260790835298)]

In [12]:
evaluation['NMF']

[(5, 0.7147033104403053),
 (10, 0.7358479263967541),
 (15, 0.7352788685093538),
 (20, 0.7255836088832905),
 (50, 0.6186516384203021)]

In [13]:
evaluation['PCA']

[(5, 0.6080252573570404),
 (10, 0.5470605282364212),
 (15, 0.4786088251630099),
 (20, 0.43360202923035673),
 (50, 0.35129601997751037)]

In [14]:
evaluation['RP']

[(5, 0.5388460051693942),
 (10, 0.5435245527262123),
 (15, 0.5373158420089561),
 (20, 0.5206040179267396),
 (50, 0.5219495119538514)]

Now we'll fit the LDA model with the number of topics that yields the highest coherence

In [ ]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5,
                     alpha='symmetric', eta='auto', passes=5, random_state=1)

In [ ]:
for topic in lda_model.print_topics(num_words=15):
    topic_index, words = topic
    word_list = [word.split("*")[1].strip().strip('"') for word in words.split(" + ")]
    print(f"Topic {topic_index}: {', '.join(word_list)}")

In [ ]:
import pyLDAvis, pyLDAvis.gensim

In [ ]:
# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='tsne')
pyLDAvis.display(vis)

Now we're going to do the same for LSA

In [ ]:
lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=5)

In [ ]:
for topic in lsi_model.print_topics():
    topic_index, words = topic
    word_list = [word.split("*")[1].strip().strip('"') for word in words.split(" + ")]
    print(f"Topic {topic_index}: {', '.join(word_list)}")